# Gaussian Process Regression

**Dan Foreman-Mackey** (danfm@nyu.edu)

In this lab, we'll work through an example problem in Gaussian process regression using Python.
The goal of the lab is to give you a qualitative sense of how Gaussian process regression works and a rough idea of how you might implement it in a real world problem.
We'll get our feet wet by fitting a linear model to data with correlated noise and a small number of data points where performance isn't a huge limiting factor.

The lab is designed to be open-ended and the last sections should be instructive even for students who are already familiar with the implementation of Gaussian processes.
I've chosen to use Python for the lab because it is my programming language of choice and because it is becoming commonly used in astronomy.
If you don't have experience coding in Python I've tried to include a lot of skeleton code so you should still be able to make progress without getting too caught up with Python's funny syntax.
Also, if you ever find yourself writing a `for` loop then something is wrong; Python is an interpreted language and, as a result, looping is *very* slow.
This entire lab can be done without any looping.

*Remember*: The best reference for all things Gaussian process is [Rasmussen & Williams](http://www.gaussianprocess.org/gpml/).

## Technical details

If you're reading this then you've probably already figured out how to run this IPython notebook and interact with it via your browser.
The entire lab will be done in this notebook but a lot of code is implemented in a Python module called `gp`; you can see [the source code on GitHub](https://github.com/dfm/gp/tree/master/gp).

You should be able to execute all the code blocks in this notebook (by clicking on it and typing `Shift-Enter`) and the results will appear below.
In a few cells, you'll need to implement some code.
To do that, you might need to double-click in the cell to enter "insert" mode (the cell will be highlighted in green or blue).
When you finish editing the cell, type `Shift-Enter` to exit insert mode and exucute what you've written.
For every function that you have to write, there will be an associated unit test that checks your implementation to make sure that it's correct (i.e. consistent with my version).
There is also one section where the notebook will offer you some sliders to interact with the figures.

If you have any issues, **don't hesitate to ask questions**!
This lab is not meant to test your Python skillz and I'm happy to help with any implementation issues that you run into.

Let's get started...

## Fitting a line to data with correlated noise

I've generated some data from a linear model with a non-trivial observation-observation covariance matrix.
The $x$ (e.g., time of a measurement, assumed to be known precisely) and $y$ (e.g., the measured brightness of a source) coordinates are reported along with the independent component of the measurement variance.
This is a pretty realistic scenario where we have a good "physical" model of the mean behaviour of the data and estimates of the per-observation measurement uncertainties but no good constraint on any covariances.

We'll start by fitting the data assuming that the observations are uncorrelated and then we'll build (and fit) a Gaussian process model with a flexible kernel function.

### Getting started

To start, let's load the data and plot it.
I've included a Python module (creatively called `gp`) that includes all of the code that you'll need to interact with the data.
Let's start by importing this module (and everything else that we'll need below) to make sure that everything is working.
Select the cell below and type `Shift-Enter` to execute the contents:

In [ ]:
%matplotlib inline

# Start by importing the specific code for this project.
import gp

# And then the standard numerical Python modules.
import numpy as np
import matplotlib.pyplot as pl

# Finally set up the plotting to look a little nicer.
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100

If that worked, then you shouldn't get any errors (or other messages) when you execute the previous line.
If that's the case, we can load the data file and plot it (you should, again, select the following cell and type `Shift-Enter` again):

In [ ]:
x, y, yerr = gp.line.load_data()
gp.line.plot_data(x, y, yerr);

Do you see a plot there with error bars and a line?
The points with error bars are the data that I generated and the line is the true mean model that I used to generate these data.
The code used to generate this plot isn't complicated and you can check it out on [GitHub](https://github.com/dfm/gp/blob/master/gp/line/plotting.py) but we're going to focus on the fitting and not the plotting for today.

Great! Let's start fitting.

### Least-squares fit

Any self-respecting astronomer would look at these data, squint, and then fit a line to them assuming that the given uncertainties are correct and the noise is uncorrelated.
This is not a generally bad idea so let's do that here.
To do this, we could use numpy's [`polyfit`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) function but they use a [strange definition for the resulting covariance](https://github.com/numpy/numpy/blob/master/numpy/lib/polynomial.py).
Instead, let's use the simple [`gp.line.least_squares`](https://github.com/dfm/gp/blob/master/gp/line/lls.py) function that I wrote:

In [ ]:
mu, cov = gp.line.least_squares(x, y, yerr)

This gives us the mean and covariance matrix of the 2-D posterior constraint on the slope and intercept of the line under the assumption of known independent uncertainties and uniform priors on $m$ and $b$.

There is a function for plotting the results (called [`gp.line.plot_results`](https://github.com/dfm/gp/blob/master/gp/line/plotting.py)) but, for reasons that you'll see below, it is designed to work with posterior samples.
Therefore, we'll first convert our Gaussian posterior constraint to a list of slope and intercept samples and then plot the results.

In [ ]:
ls_samples = np.random.multivariate_normal(mu, cov, 20000)
gp.line.plot_results(x, y, yerr, ls_samples);

The previous command should give you two plots.
The first shows an estimate of the 68% credible interval for the posterior predictive distribution (in red) plotted over the data (points with error bars) and compared to the Truth (black line).
In other words, the lower (upper) red line is the 16th (84th) percentile of the distribution for the predictions of the current model.
The second figure shows the "corner plot" of the posterior samples plotted in parameter space.
The true values are

$$m = 0.5 \quad \mathrm{and} \quad b = -0.25 \quad$$

and they are so far off the second plot that they aren't even visible.
This problem is also apparent on the first figure where the posterior prediction is extremely precise (the error bars are tiny!) but the true line is very strongly excluded.

The problem is that the noise in the data is *correlated* but the fit we calculated assumed that the noise was uncorrelated.
We'll fix this problem in a moment but first an aside...

### Building a Gaussian process model

#### The likelihood function

Now let’s implement the code for a Gaussian process.
The main thing that you need to do in this section is write the ln-likelihood function.
What you need to do is convert the following mathematical equation (you might remember it from lecture) to Python:

$$
\ln L + C = -\frac{1}{2}\,\boldsymbol{r}^\mathrm{T}\,K^{-1}\,\boldsymbol{r} - \frac{1}{2}\,\ln \det \boldsymbol{K}
$$

where $C = N\,\ln(2\,\pi)/2$ is an irrelevant (for our purposes) constant, $\boldsymbol{r}$ is the residual vector (the difference between the measured $y$ and the model predictions for $y$), and $K$ is the covariance matrix describing the measurement uncertainties and the correlations between observations.

In the following cell, you'll implement a Python function that takes as input a residual vector `r` and a covariance matrix `K` and returns the right-hand side of the above equation.
In pseudocode, the function will look something like:

<hr>

**Input:** Residual vector $\boldsymbol{r}$; covariance matrix $K$<br>
**Output:** The ln-likelihood (up to a constant)
1. $\boldsymbol{\alpha} = \mathtt{np.linalg.solve}({K},\,\boldsymbol{r})$
2. $(s,\,d) = \mathtt{np.linalg.slogdet}(\boldsymbol{K})$
3. **return** $-0.5\,\left(\boldsymbol{r}\cdot\boldsymbol{\alpha} + d\right)$

<hr>

When implementing this function, the two numpy (remember that we imported `numpy` as `np` above) methods used in the pseudocode ([`np.linalg.solve`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html) and [`np.linalg.slogdet`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.slogdet.html)) will come in handy.
The `solve` method is useful because you should *never* invert a matrix (it is numerically unstable).
Instead, you should always use `solve` if you want to find $\boldsymbol{x}$ in the equation:

$$
\boldsymbol{x} = {K}^{-1}\,\boldsymbol{r} \to \boldsymbol{r} = {K}\,\boldsymbol{x} \quad.
$$

The `slogdet` function is useful because it computes the ln-determinant of a matrix using a numerically stable method.
Determinants of physical matrices tend to be *very* tiny and computing them naively can result in numerical underflow.
Sticking to the ln-determinant will make you less suceptible to this problem.

Finally, if you want to take the dot product of two vectors or matrices, you should use the [`np.dot`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) function.

OK. Go for it...

In [ ]:
def lnlike(r, K):
    """
    The multivariate Gaussian ln-likelihood (up to a constant) for the
    vector ``r`` given a covariance matrix ``K``.
    
    :param r: ``(N,)``   The residual vector with ``N`` points.
    :param K: ``(N, N)`` The square (``N x N``) covariance matrix.
    
    :returns lnlike: ``float`` The Gaussian ln-likelihood. 
    
    """
    # Erase the following line and implement the Gaussian process
    # ln-likelihood here.
    pass

When you're happy with your implementation, execute the following cell.
If your implementation is correct, you'll see a smiley face (☺︎) otherwise, it will throw an exception.

In [ ]:
gp.utils.test_lnlike(lnlike)

Did you see a happy face when you executed the above line?
Edit your `lnlike` function until you do and then let's move on to implementing a specific model.

#### The covariance matrix/kernel function

Now that we have the likelihood function implemented, we need to choose a specific model of the covariance matrix $K$.
Each element of the matrix $K_{ij}$, will be given by

$$
K_{ij} = k_{\boldsymbol{\alpha}}(x_i,\,x_j)
$$

where we get to choose the "kernel function" $k_{\boldsymbol{\alpha}}$ (parameterized by ${\boldsymbol{\alpha}}$).
For now, let's implement the *exponential-squared kernel*:

$$
k_{\boldsymbol{\alpha}}(r_{ij}) = a^2 \, \exp \left ( -\frac{{\Delta x_{ij}}^2}{2\,l^2} \right )
$$

where $\boldsymbol{\alpha} = (a,\,l)$ and $\Delta x_{ij} = |x_i - x_j|$.

You need to implement this function below and it needs to work with a `numpy` array `dx` as input (use the `np.exp` function for exponentiation).
Note that `alpha` will be a list with two elements: `alpha[0] = amp` and `alpha[1] = ell`; Python arrays are zero-indexed.

In [ ]:
def expsq_kernel(alpha, dx):
    """
    The exponential-squared kernel function. The difference matrix
    can be an arbitrarily shaped numpy array so make sure that you
    use functions like ``numpy.exp`` for exponentiation.
    
    :param alpha: ``(2,)`` The parameter vector ``(amp, ell)``.
    :param dx: ``numpy.array`` The difference matrix. This can be
        a numpy array with arbitrary shape.
    
    :returns K: The kernel matrix (should be the same shape as the
        input ``dx``). 
    
    """
    # Erase the following line and implement your kernel function
    # there.
    pass

Once you're happy with your implementation of `expsq_kernel`, try executing the following cell to test your code.
Again, if you've implemented it correctly, you'll see a smiley face.

In [ ]:
gp.utils.test_kernel(expsq_kernel)

### Qualitative effects of the kernel choice

Now that we've chosen (and implemented) our kernel function, let's try to get a qualitative sense of what the hyperparameters do.

To start, we'll generate samples from the likelihood function that we implemented.
Remember that a likelihood is a probability distribution over *data* so, conditioned on specific hyperparameters, we can sample synthetic datasets from the likelihood.
For a Gaussian process, this probability function is a multivariate Gaussian where the covariance matrix is given by our kernel function.

In the following cell, we're using some IPython magic to create an interactive figure that displays 6 samples $\boldsymbol{y}_k$ from the likelihood for variable values of the hyperparameters.
Try executing the cell and then moving the sliders to get a feel for how the samples change with $a$ (`amp`) and $l$ (`ell`).

In [ ]:
gp.line.interactive.setup_likelihood_sampler(expsq_kernel);

More interesting is the predictive distribution.
That is, given the data that you have observed and a specific set of parameters, what is your prediction at other points $\boldsymbol{x}_\star$?
The math is in my lecture slides but for now, you can just execute the following cell and play around with the parameters to see how the predictions change with all the parameter choices.

In the figure, the "noiseless" mean linear function is shown as a dotted red line and the 6 samples from the conditional distribution are shown as black lines.
The ln-likelihood is shown in the title of the figure.
We'll use the results of your experiments to initialize the MCMC sampler in a bit so try to get a good fit.

In [ ]:
w = gp.line.interactive.setup_conditional_sampler(x, y, yerr, expsq_kernel);

### Benchmarking

Let’s take a moment now to benchmark our code and see how each component scales with the size of the dataset.
There is a function included in the `gp` module that runs a bunch of tests with datasets of different sizes and plots the results.
To benchmark your implementation, you can call this function as follows:

In [ ]:
hyper = [1.0, 0.5]  # The zeroth element is `amp` and the first is `ell`.
N, t = gp.benchmark(expsq_kernel, lnlike, hyper, N=2 ** np.arange(8, 13))

The returned arrays `N` and `t` give the number of datapoints used and the time taken (in seconds) for the various components of the process:

* `kernel`: the time taken to build the covariance matrix,
* `lnlike`: the time taken to compute the ln-likelihood, and
* `total`: the total time for the computation.

The figure produced should also show (in square brackets in the legend) the empirical scaling (computed as a power-law fit) of the time required for each component with $N$.
For example, the theoretical scaling of the kernel build should be $\mathcal{O}(N^2)$ and the computation of the ln-likelihood should be $\mathcal{O}(N^3)$.

**Questions:**
Is this what you get?
Why or why not?
Try playing around with the hyperparameter choices to see how the scaling changes.

**Optional:**
If you're so inclined, now might be a good time to try to make your code a little faster.
It might be hard to improve the kernel build phase but you can improve the constant factor in the ln-likelihood computation by using scipy's [`cho_factor`](http://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.linalg.cho_factor.html) and [`cho_solve`](http://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.linalg.cho_solve.html) functions.
These functions can help because both `solve` and `slogdet` must factor the kernel matrix before computing their solutions.
Instead you can just compute the Cholesky factor $L$ once (where $K = L\,L^\mathrm{T}$) and use the fact that $\ln\det K = 2\,\ln\det L = 2\,\sum_n \ln L_{nn}$.

*Hints:*  Follow the links to documentation for `cho_factor` to see what, exactly, it returns.
You can extract the diagonal elements from a matrix using the [`np.diag`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.diag.html) function.
Other functions you may want to use include `np.sum` and `np.log`.

In [ ]:
# Optionally implement a fast ln-likelihood function.

from scipy.linalg import cho_factor, cho_solve

def faster_lnlike(r, K):
    """
    A faster version of ``lnlike``.
    
    """
    # Erase the following line and implement the Gaussian process
    # ln-likelihood here.
    pass

# Check the implementation...
gp.utils.test_lnlike(faster_lnlike)

Once you have a working implementation of the faster likelihood function (and you see a smiley face) run the benchmark using this function:

In [ ]:
faster_N, faster_t = gp.benchmark(expsq_kernel, faster_lnlike, hyper, N=2 ** np.arange(8, 13))

If everything went as planned, your faster implementation should be a factor of a few (2-4ish) faster for every value of $N$:

In [ ]:
t["lnlike"] / faster_t["lnlike"]

### Fitting the data using a Gaussian process

We're finally at the point where we can fit the data using a Gaussian process.
Since we don't have any strong reason to adopt a specific setting for the hyperparameters, we'll apply uniform priors on all the parameters (uniform in the *log* of the hyperparameters) and use MCMC (implemented using the Python package [`emcee`](http://dan.iel.fm/emcee)) to sample from the joint posterior probability:

$$
p(m,\,b,\,a,\,l\,|\,\boldsymbol{y},\,\boldsymbol{x},\,\boldsymbol{\sigma}^2)
$$

The `gp` module contains code for doing this sampling (with sane choices of defaults).

We need to start by choosing an initial point.
We'll use the values that you chose in the interactive plot above (so make sure that they are good choices):

In [ ]:
initial = [w.kwargs[k] for k in ["m", "b", "amp", "ell"]]

Then, let's sample using the built-in MCMC function (change `lnlike` to `faster_lnlike` if you implemented the faster version in the optional section above).
Using the default settings and a reasonably efficient `lnlike` implmentation, this might take up to 5 minutes.
Feel free to experiment by varying the number of burn-in steps (the `burnin` argument) and the number of production steps (`steps`).

In [ ]:
samples = gp.line.sample(x, y, yerr, expsq_kernel, lnlike, initial, steps=2000, burnin=2000, thin=37)

And then, we'll use the same function (`gp.line.plot_results`) as we used above to display our results.
For comparison, we'll also show the results of the least-squares version.

In [ ]:
data_fig, _ = gp.line.plot_results(x, y, yerr, ls_samples, show=False)
gp.line.plot_results(x, y, yerr, samples, data_fig=data_fig, color="g");

The first results plot shows the marginalized posterior constraints plotted (in green) on top of the data points and compares this to the least-squares solution (shown in red).
The triangle plot shows the results of the MCMC sampling plotted in all the possible projections and these are compared to the true values (shown as blue lines).

**Questions:**
How does your result compare to the result obtained using least squares (assuming that the uncertainties are known and independent)?
Are your results consistent with the truth?
Why or why not?
Do you think the MCMC sampling has converged and how would you test your hypothesis?
(You don't need to implement this, just think about some convergence tests you could try.)

It might be helpful to examine some quantiles of the posterior samples.
The following snippet of code finds the 0.16, 0.5, and 0.84 sample quantiles (roughly 68% of the probability mass) and compares them to the true values.

In [ ]:
from IPython.display import display, Math, Latex

quantiles = np.array(zip(*(np.percentile(samples, [16, 50, 84], axis=0))))
truth = gp.utils.load_data("line_true_params.txt")
truth[2:] = np.log(truth[2:])
fmt = r"\mathrm{{Param}}\,{0}:\quad \mathrm{{truth}} = {1:.4f};\quad\mathrm{{fit}} = {2:.4f}_{{-{3:.4f}}}^{{+{4:.4f}}}"

for i, (q, t) in enumerate(zip(quantiles, truth)):
    display(Math(fmt.format(i+1, t, q[1], *(np.diff(q)))))

At this point, we've finished the key components of this lab but if you have time left (or if you just can't get enough and want to continue at home), you can continue on to the next section where there are some suggestions of other things to try with this dataset.

## Optional: Where to go from here

### 1. Different kernels

At the beginning of the lab, we chose to use the exponential-squared kernel function for all of our experiements.
This was a good idea because it is *exactly* the kernel that I used to generate the data.
That being said, in The Real World™, we never know what kernel to use and there are some other commonly used functions.
One good simple example is the Matern-3/2 function

$$
k_{\boldsymbol{\alpha}}(r_{ij}) = a^2 \, \left[1+\frac{\sqrt{3}\,|\Delta x_{ij}|}{l}\right]\, \exp \left ( -\frac{\sqrt{3}\,|\Delta x_{ij}|}{l} \right )
$$

Try implementing this function (or another one of your choice; see [Chapter 4](http://www.gaussianprocess.org/gpml/chapters/RW4.pdf) of [R&W](http://www.gaussianprocess.org/gpml/) for examples) and re-run the analysis with this kernel.
There is no included unit test for this kernel (since this is an optional extension) but you might consider making some plots or writing your own tests to make sure that the function returns sensible values.

In [ ]:
def matern32_kernel(alpha, dx):
    """
    The Mater-3/2 kernel function. The difference matrix
    can be an arbitrarily shaped numpy array so make sure that you
    use functions like ``numpy.exp`` for exponentiation.
    
    :param alpha: ``(2,)`` The parameter vector ``(amp, ell)``.
    :param dx: ``numpy.array`` The difference matrix. This can be
        a numpy array with arbitrary shape.
    
    :returns K: The kernel matrix (should be the same shape as the
        input ``dx``). 
    
    """
    # Erase the following line and implement your kernel function
    # there.
    pass

Take a look at the conditional distribution given by this new choice of kernel:

In [ ]:
w_matern = gp.line.interactive.setup_conditional_sampler(x, y, yerr, matern32_kernel);

Try running the MCMC sampling with this kernel (change `lnlike` to `faster_lnlike` if applicable)...

In [ ]:
initial_matern = [w_matern.kwargs[k] for k in ["m", "b", "amp", "ell"]]
samples_matern = gp.line.sample(x, y, yerr, matern32_kernel, lnlike, initial_matern, steps=2000, burnin=2000, thin=37)

...and look at the results.

In [ ]:
data_fig, _ = gp.line.plot_results(x, y, yerr, ls_samples, show=False)
gp.line.plot_results(x, y, yerr, samples, data_fig=data_fig, color="g", show=False)
gp.line.plot_results(x, y, yerr, samples_matern, data_fig=data_fig, color="b");

**Questions**:
How do the new results (plotted in blue) compare to the previous results?
How do you explain the differences?

Try plotting the value of $k(\Delta x)$ as a function of $\Delta x$ for the two different kernels.
What do you see?
How would this lead to the different results that you found?

### 2. Priors on hyperparameters

In the previous examples, we've been using uniform priors on the logarithm of the hyperparameters; you can also try using a uniform prior on the *linear* hyperparameters by setting the argument `logunifhyper` to `False`.

In [ ]:
samples_linear = gp.line.sample(x, y, yerr, expsq_kernel, lnlike, initial,
                                steps=4000, burnin=4000, thin=37, logunifhyper=False)

In [ ]:
data_fig, _ = gp.line.plot_results(x, y, yerr, ls_samples, show=False)
gp.line.plot_results(x, y, yerr, samples, data_fig=data_fig, color="g", show=False)
gp.line.plot_results(x, y, yerr, samples_linear, data_fig=data_fig, color="b");

**Questions:**
Why does do the constraints look so different?
Can you explain the covariances?
Why does the prior have such a strong effect?